In [20]:
import pandas as pd

from scanning_window import scan_record
from read_record import Record,RecordReader

from collections import Counter

In [2]:
folder_path="D:/CPSC2021/Training_set_I"
file_name="data_1_7"

In [17]:
reading_record=RecordReader.read(folder_path,file_name,0,0,None)

In [18]:
# Get the keys of the object
keys=reading_record.__dict__.keys()

# Convert the keys to a list if you need
keys_list = list(keys)

# Print the keys
print(keys_list)

['_Record__parent', '_Record__signal', '_Record__symbol', '_Record__aux', '_Record__sample', '_Record__label', '_Record__sf', '_Record__collection']


In [19]:
ecg_signal=reading_record._Record__signal
ecg_signal

array([-0.00120646, -0.00241292,  0.00422261, ..., -0.00693715,
       -0.00663554, -0.01146138])

In [35]:
list(Counter(reading_record._Record__aux).keys())==['']

True

In [31]:
# Check if a string is empty
my_string = ''
if my_string == '':
    print("The string is empty.")
else:
    print("The string is not empty.")


The string is empty.
